In [348]:
from core import database
import isb
import pandas as pd
import numpy as np
import h3
import typing as t
import plotly.express as px
import plotly.graph_objects as go

mapbox_access_token = open(".mapbox_token").read()

In [349]:
db = database()

C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 



Task exception was never retrieved
future: <Task finished name='Task-11293' coro=<RefreshAheadCache._schedule_refresh.<locals>._refresh_task() done, defined at C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py:142> exception=RefreshError('Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.')>
Traceback (most recent call last):
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py", line 152, in _refresh_task
    refresh_data = await refresh_task
                   ^^^^^^^^^^^^^^^^^^
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py", line 107, in _perform_refresh
    connection_info = await self._client.get_connection_info(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\

In [350]:
df_listings = db.query("""
select 
  r.realestate_id, 
  l.listing_id,
  l.span as listing_span, 
  lp.span as listing_price_span, 
  u.district_id,
  r.square_meters, 
  r.rooms,
  r.construction_year,
  rt.name as realestate_type_name, 
  case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
  c.name as city_name,
  a.name as address_name,
  a.street_name,
  case when construction_year >= extract(year from now()) - 5 and (sa.first_sale_date is null or sa.first_sale_date >= now()) then 'new' else 'old' end as construction_category,
  price,
  (now()::date - lower(l.span)::date)::int as days_listed,
  ST_X(u.origin) as latitude, 
  ST_Y(u.origin) as longitude
from listings l
join listing_prices lp
on l.realestate_id = lp.realestate_id
and l.span && lp.span
join realestates r
on lp.realestate_id = r.realestate_id
left join (
  select realestate_id, min(registration_date) as first_sale_date
  from sale_agreements
  group by realestate_id         
) sa
on r.realestate_id = sa.realestate_id
join realestate_types rt
on r.realestate_type_id = rt.realestate_type_id
join units u 
on r.unit_id = u.unit_id
join addresses a
on u.address_id = a.address_id
join lands la
on a.land_id = la.land_id
join postals p
on la.postal_id = p.postal_id
join cities c
on p.city_id = c.city_id
join regions reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'
  and now()::date <@ lp.span
""")\
.assign(sqm_price = lambda r: r.price / r.square_meters)

['makro-website:europe-west2:makro-prod']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py", line 152, in _refresh_task
    refresh_data = await refresh_task
                   ^^^^^^^^^^^^^^^^^^
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\instance.py", line 107, in _perform_refresh
    connection_info = await self._client.get_connection_info(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\cloud\sql\connector\client.py", line 274, in get_connection_info
    self._credentials.refresh(requests.Request())
  File "C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\oauth2\credentials.py", line 409, in refresh
    ) = reauth.refresh_grant(


In [351]:
df_sale_agreement_times = db.query("""
select 
    sa1.sale_agreement_id, 
    sum(
        least(sa1.registration_date, upper(l1.span))::date -
        least(sa1.registration_date, lower(l1.span))::date
    ) as listing_days
from listings l1
join sale_agreements sa1
on l1.realestate_id = sa1.realestate_id
and l1.span && daterange((registration_date - make_interval(months:=12))::date, registration_date::date, '[]')
and '2025-01-01' <= sa1.agreement_date    
group by sa1.sale_agreement_id
""")

In [352]:
df_sale_agreement_times

,sale_agreement_id,listing_days
0,00100cde-23f0-423e-985b-39dfa12da341,279
1,0014059e-447f-414a-a69b-3b0d84c9beab,53
2,00177410-db17-4bd5-97c2-c80d15e250bb,140
3,0019f0b0-eb5a-4cf3-afbb-a6f0abfa99b2,5
4,002122f5-4fad-4f2a-93ec-d9690c2c26d9,81
...,...,...
8526,ffe4fb66-1f86-4686-a245-7c9cd6ed15c6,65
8527,ffeecd55-2de4-424f-a9f1-929984090384,49
8528,fff7c9de-7b82-413b-ba3d-e81a8e36cb49,23
8529,ffff2615-e521-42be-9b29-d2348f4ce2a5,176


In [353]:
df_sales = db.query("""
select 
  s.sale_agreement_id,
  r.realestate_id, 
  u.district_id,
  r.square_meters, 
  r.rooms,
  r.construction_year,
  rt.name as realestate_type_name, 
    case when rt.name in ('Fjölbýlishús', 'Hæð') then 'Fjölbýli' else 'Sérbýli' end as realestate_category_name,
  c.name as city_name,
  a.name as address_name,
  a.street_name,
  case when construction_year >= extract(year from now()) - 5 and (sa.first_sale_date is null or sa.first_sale_date < s.registration_date) then 'new' else 'old' end as construction_category,
  purchase_price, 
  (select price from listing_prices lp where lp.span && daterange((s.registration_date - make_interval(months:=12))::date, s.registration_date::date, '[]') and lp.realestate_id = s.realestate_id order by span desc limit 1) as listing_price,
  --ld.listing_days,
  ST_X(u.origin) as latitude, 
  ST_Y(u.origin) as longitude
from sale_agreements s
join realestates r
on s.realestate_id = r.realestate_id
left join (
  select realestate_id, min(registration_date) as first_sale_date
  from sale_agreements
  group by realestate_id         
) sa
on r.realestate_id = sa.realestate_id
join realestate_types rt
on r.realestate_type_id = rt.realestate_type_id
join units u 
on r.unit_id = u.unit_id
join addresses a
on u.address_id = a.address_id
join lands la
on a.land_id = la.land_id
join postals p
on la.postal_id = p.postal_id
join cities c
on p.city_id = c.city_id
join regions reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'
  and '2025-01-01' <= s.agreement_date    
  and is_valid_agreement             
""")\
.assign(sqm_price = lambda r: r.purchase_price / r.square_meters)\
.merge(df_sale_agreement_times, how='left')

In [355]:
def get_sales_filtered_by(realestate_category_name: str='Fjölbýli', construction_category: str='new', city_name: str=''):
    return df_sales\
        .loc[lambda r: r.realestate_category_name == realestate_category_name]\
        .loc[lambda r: r.construction_category == construction_category]\
        .loc[lambda r: r.square_meters >= 20]\
        .loc[lambda r: r.square_meters < 250]

def get_listings_filtered_by(realestate_category_name: str='Fjölbýli', construction_category: str='new', city_name: str=''):
    return df_listings\
        .loc[lambda r: r.realestate_category_name == realestate_category_name]\
        .loc[lambda r: r.construction_category == construction_category]\
        .loc[lambda r: r.square_meters >= 20]\
        .loc[lambda r: r.square_meters < 250]

In [356]:
fig = isb.Figure()

df_hfj_sale = get_sales_filtered_by(city_name='Reykjavíkurborg')

fig.add_scatter(
    y = df_hfj_sale.city_name.apply(lambda x: 1.1 + np.random.random()/100),
    x = df_hfj_sale.square_meters,
    mode = 'markers',
    marker = dict(
        color = fig.colors(1)
    )
)
df_hfj_list = get_listings_filtered_by(city_name='Reykjavíkurborg')

fig.add_scatter(
    y = df_hfj_list.city_name.apply(lambda x: 1.1 + np.random.random()/100),
    x = df_hfj_list.square_meters,
    mode = 'markers',
    marker = dict(
        color = fig.colors(0)
    )
)

In [357]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

# Example: your pandas series
sqm = df_hfj_list.square_meters


def fit_dist(values: pd.Series, x: np.array, print_results: bool=False) -> pd.DataFrame:

    distributions = {
        "normal": stats.norm,
        "lognorm": stats.lognorm,
        "gamma": stats.gamma,
        "weibull": stats.weibull_min
    }

    results = {}

    for name, dist in distributions.items():
        # Fit distribution to data
        params = dist.fit(values)
        
        # Compute log-likelihood
        loglik = np.sum(dist.logpdf(sqm, *params))
        
        # Calculate AIC = 2k - 2logL
        k = len(params)
        aic = 2 * k - 2 * loglik

        results[name] = {"params": params, "loglik": loglik, "aic": aic}

    best_dist = min(results, key=lambda x: results[x]["aic"])

    if print_results:
        print("Best fitting distribution:", best_dist)
        print("Parameters:", results[best_dist]["params"])

    dist = distributions[best_dist]
    params = results[best_dist]["params"]

    pdf = dist.pdf(x, *params)

    return pd.DataFrame(dict(
        x = x,
        pdf = pdf
    ))

In [358]:

def make_plot(df, title="Hlutfall eigna <color 0>á söluskrá</color> og <color 2>seldra</color> (ekki nýbyggingar)"):

    fig = isb.Figure('core')

    xs = df.index.to_series().to_list()[1:-1]

    for i, column in enumerate(df.columns):

        color_i = i*2 + 1

        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(color_i),
                width = 3
            ),
            fillcolor = fig.colors(color_i, .20),
            fill = 'tozeroy',
            showlegend=False
        )

        xs = df.index.to_series()
        ys = df[column]

        average = np.average(xs, weights=ys)
        position = df.loc[round(average)]

        fig.add_scatter(
            x = [average]*2,
            y = [0, position[column]],
            mode = 'lines',
            line = dict(
                color = fig.colors(color_i),
                dash = 'dash',
                width = 3
            ),
            showlegend=False
        )

        names = dict(
            pdf_list = f"Eignir á sölu<br><b>{round(average, 1)} m²</b><br>að meðaltali",
            pdf_sale = f"Seldar eignir<br><b>{round(average, 1)} m²</b><br>að meðaltali"
        )

        fig.add_annotation(
            x = average,
            y = position[column],
            text = names[column],
            showarrow=True,
            arrowsize=2,
            ax = 30 * (1 if i == 0 else -1),
            ay = -80,
            yshift=2.5,
            bgcolor = fig.bg_color,
            arrowcolor=fig.font_color,
            font = fig.get_font(size=16)
        )

        
        fig.add_scatter(
            x = [average],
            y = [position[column]],
            mode = 'markers',
            marker = dict(
                color = fig.colors(color_i),
                size=8
            ),
            showlegend=False
        )
        fig.add_scatter(
            x = [average],
            y = [position[column]],
            mode = 'markers',
            marker = dict(
                color = fig.bg_color,
                size=4
            ),
            showlegend=False
        )


        print(np.average(xs, weights=ys))

    fig.update_yaxes(
        tickformat = '.1%'
    )

    
    fig.set_title(
        title,
    )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = "<i>Sölur miðast við allar sölur á árinu 2025, eignir á söluskrá miðast við 23. nóvember 2025</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    fig.add_logo()

    return fig


df_hfj_sale = get_sales_filtered_by(construction_category='new')
df_hfj_list = get_listings_filtered_by(construction_category='new')

fit_dist(df_hfj_list.square_meters, x=np.arange(0, 200, 1)).set_index('x')\
.merge(
    fit_dist(df_hfj_sale.square_meters, x=np.arange(0, 200, 1)).set_index('x'),
    right_index=True, left_index=True,
    suffixes=["_list", "_sale"]
)\
.pipe(make_plot, title="Hlutfall eigna <color 1>á söluskrá</color> og <color 3>seldra</color> (aðeins nýbyggingar)")\
.export("04-sales-vs-listing--square-meters--new-construction", scale=1.5)


df_hfj_sale = get_sales_filtered_by(construction_category='old')
df_hfj_list = get_listings_filtered_by(construction_category='old')

fit_dist(df_hfj_list.square_meters, x=np.arange(0, 200, 1)).set_index('x')\
.merge(
    fit_dist(df_hfj_sale.square_meters, x=np.arange(0, 200, 1)).set_index('x'),
    right_index=True, left_index=True,
    suffixes=["_list", "_sale"]
)\
.pipe(make_plot, title="Hlutfall eigna <color 1>á söluskrá</color> og <color 3>seldra</color> (ekki nýbyggingar)")\
.export("04-sales-vs-listing--square-meters--old-construction", scale=1.5)

104.50834061723035
88.75816422458254
103.2177412415307
96.97472861626257


In [359]:

df_hfj_sale = get_sales_filtered_by(construction_category='new')
df_hfj_list = get_listings_filtered_by(construction_category='new')


fit_dist(df_hfj_list[(df_hfj_list.sqm_price > 300e3) & (df_hfj_list.sqm_price < 2000e3)].sqm_price, x=np.arange(0, 1.5e6, 10e3)).set_index('x')\
.merge(
    fit_dist(df_hfj_sale[(df_hfj_sale.sqm_price > 300e3) & (df_hfj_sale.sqm_price < 2000e3)].sqm_price, x=np.arange(0, 1.5e6, 10e3)).set_index('x'),
    right_index=True, left_index=True,
    suffixes=["_list", "_sale"]
)\
.isb.plot()

In [360]:
import statsmodels.api as sm

In [361]:
df_temp.head()

,realestate_id,listing_id,listing_span,listing_price_span,district_id,square_meters,rooms,construction_year,realestate_type_name,realestate_category_name,city_name,address_name,street_name,construction_category,price,days_listed,latitude,longitude,sqm_price
0,012a32c5-8275-4bfd-b83a-1eee1afd6b83,6c2dc688-8ec5-4066-903f-a3a4d097d6ba,"[2025-10-08,2025-11-24)","[2025-10-08,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,77.1,3.0,1939.0,Hæð,Fjölbýli,Reykjavíkurborg,Freyjugata 25,Freyjugata,old,69900000,46,-21.930513,64.142310,9.066148e+05
1,01f180f4-2bdf-4b58-97ca-a993f4f752d7,9250a118-7df0-4e46-be8b-1d493c2181da,"[2024-08-22,2025-11-24)","[2024-08-22,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,55.8,1.0,1942.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Tryggvagata 4-6,Tryggvagata,old,54500000,458,-21.944630,64.150690,9.767025e+05
2,0202839e-33cd-4caa-803c-3b8b8a7a8715,0d6afbf7-2752-491e-92f3-72a41627c5e3,"[2025-05-26,2025-11-24)","[2025-11-14,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,130.0,4.0,2019.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Frakkastígur 8C,Frakkastígur,old,149500000,181,-21.925062,64.145390,1.150000e+06
3,02d77e8f-4ad3-412e-86a6-1ece05af1eca,f5271851-9054-4c8a-8100-cc19c5db4924,"[2025-03-17,2025-11-24)","[2025-07-17,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,129.5,2.0,2019.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Reykjastræti 7,Reykjastræti,old,189000000,251,-21.935675,64.149260,1.459459e+06
4,050fbdbc-2763-4fa3-8bae-91add13b5fcc,0cb31890-7d51-49cf-99a4-b58667a3709f,"[2025-08-19,2025-11-24)","[2025-08-19,2025-11-24)",b7b8ed8c-8a80-4423-9de9-041822d94fd4,48.9,2.0,2023.0,Fjölbýlishús,Fjölbýli,Reykjavíkurborg,Mýrargata 37,Mýrargata,old,56900000,96,-21.952948,64.152115,1.163599e+06


In [362]:
df_sale_centers = df_sales\
.groupby(['city_name'])\
.agg(
    latitude = ('latitude', 'mean'),
    longitude = ('longitude', 'mean')
)

In [363]:
df_listing_centers = df_listings\
.groupby(['city_name'])\
.agg(
    latitude = ('latitude', 'mean'),
    longitude = ('longitude', 'mean')
)

In [364]:
colors = isb.plot.helpers.create_color_scaler(
    min_value=0, 
    max_value=360,
    color_low="#279638",
    color_high="#fd5a54"
)
fig = isb.Figure()

fig.add_scattermapbox(
    lat=df_listing_centers.longitude, 
    lon=df_listing_centers.latitude, 
    #z=df.days_listed.apply(lambda x: 360 if x > 360 else x),
    #radius=5,
    #opacity=0.5
)
fig.add_scattermapbox(
    lat=df_sale_centers.longitude, 
    lon=df_sale_centers.latitude, 
    #z=df.days_listed.apply(lambda x: 360 if x > 360 else x),
    #radius=5,
    #opacity=0.5
)

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        style="light",
        center=dict(
            lat=64.1024998,
            lon=-21.897828
        ),
        pitch=0,
        zoom=10
    ),
)

fig.show()

In [365]:
df_listings\
.loc[lambda r: r.square_meters < 300]\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.loc[lambda r: r.construction_category == 'new']\
.assign(sqm_price = lambda r: r.price / r.square_meters)\
.groupby(['realestate_category_name', 'city_name', 'construction_category'])\
.agg(
    square_meters = ('square_meters', 'mean'),
    rooms = ('rooms', 'mean'),
    construction_year = ('construction_year', 'mean'),
    price = ('price', 'mean'),
    total = ('realestate_id', 'count'),
    sqm_price = ('sqm_price', 'mean')
)

square_meters  \
realestate_category_name city_name               construction_category                  
Fjölbýli                 Garðabær                new                       124.804630   
                         Hafnarfjarðarkaupstaður new                        99.684987   
                         Kópavogsbær             new                       105.545536   
                         Mosfellsbær             new                        93.566667   
                         Reykjavíkurborg         new                       105.063288   
                         Seltjarnarnesbær        new                       116.186667   

                                                                           rooms  \
realestate_category_name city_name               construction_category             
Fjölbýli                 Garðabær                new                    3.250000   
                         Hafnarfjarðarkaupstaður new                    3.122137   
                         Kópavogsbær             new                    3.312500   
                         Mosfellsbær             new                    3.000000   
                         Reykjavíkurborg         new                    3.030137   
                         Seltjarnarnesbær        new                    3.333333   

                                                                        construction_year  \
realestate_category_name city_name               construction_category                      
Fjölbýli                 Garðabær                new                          2024.731481   
                         Hafnarfjarðarkaupstaður new                          2023.870229   
                         Kópavogsbær             new                          2024.660714   
                         Mosfellsbær             new                          2024.000000   
                         Reykjavíkurborg         new                          2024.369863   
                         Seltjarnarnesbær        new                          2024.466667   

                                                                               price  \
realestate_category_name city_name               construction_category                 
Fjölbýli                 Garðabær                new                    1.262280e+08   
                         Hafnarfjarðarkaupstaður new                    8.012583e+07   
                         Kópavogsbær             new                    1.068731e+08   
                         Mosfellsbær             new                    8.023333e+07   
                         Reykjavíkurborg         new                    1.119675e+08   
                         Seltjarnarnesbær        new                    1.276867e+08   

                                                                        total  \
realestate_category_name city_name               construction_category          
Fjölbýli                 Garðabær                new                      108   
                         Hafnarfjarðarkaupstaður new                      393   
                         Kópavogsbær             new                      112   
                         Mosfellsbær             new                        6   
                         Reykjavíkurborg         new                      365   
                         Seltjarnarnesbær        new                       15   

                                                                           sqm_price  
realestate_category_name city_name               construction_category                
Fjölbýli                 Garðabær                new                    1.007487e+06  
                         Hafnarfjarðarkaupstaður new                    8.111443e+05  
                         Kópavogsbær             new                    1.008830e+06  
                         Mosfellsbær             new                    8.608889e+05  
                         Reykjavíkurborg         new             

In [366]:
df_sales\
.loc[lambda r: r.square_meters < 300]\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.assign(sqm_price = lambda r: r.purchase_price / r.square_meters)\
.groupby(['realestate_category_name', 'city_name', 'construction_category'])\
.agg(
    square_meters = ('square_meters', 'mean'),
    rooms = ('rooms', 'mean'),
    construction_year = ('construction_year', 'mean'),
    purchase_price = ('purchase_price', 'mean'),
    total = ('realestate_id', 'count'),
    sqm_price = ('sqm_price', 'mean'),
    listing_days = ('listing_days', 'mean')
)

square_meters  \
realestate_category_name city_name               construction_category                  
Fjölbýli                 Garðabær                new                        97.898333   
                                                 old                       110.569042   
                         Hafnarfjarðarkaupstaður new                        99.128571   
                                                 old                        97.598498   
                         Kópavogsbær             new                        95.648276   
                                                 old                       103.020554   
                         Mosfellsbær             new                        90.305000   
                                                 old                        98.527119   
                         Reykjavíkurborg         new                        81.536478   
                                                 old                        96.818175   
                         Seltjarnarnesbær        new                       110.975000   
                                                 old                       114.036585   

                                                                           rooms  \
realestate_category_name city_name               construction_category             
Fjölbýli                 Garðabær                new                    2.933333   
                                                 old                    3.316953   
                         Hafnarfjarðarkaupstaður new                    3.222222   
                                                 old                    3.174174   
                         Kópavogsbær             new                    2.948276   
                                                 old                    3.323615   
                         Mosfellsbær             new                    2.650000   
                                                 old                    3.203390   
                         Reykjavíkurborg         new                    2.754717   
                                                 old                    3.262914   
                         Seltjarnarnesbær        new                    3.750000   
                                                 old                    3.487805   

                                                                        construction_year  \
realestate_category_name city_name               construction_category                      
Fjölbýli                 Garðabær                new                          2021.916667   
                                                 old                          2014.882064   
                         Hafnarfjarðarkaupstaður new                          2022.047619   
                                                 old                          1998.277778   
                         Kópavogsbær             new                          2021.465517   
                                                 old                          1995.696793   
                         Mosfellsbær             new                          2020.550000   
                                                 old                          2009.214689   
                         Reykjavíkurborg         new                          2021.716981   
                                                 old                          1975.361358   
                         Seltjarnarnesbær        new                          2024.500000   
                                                 old                          1988.036585   

                                                                        purchase_price  \
realestate_category_name city_name               construction_category                   
Fjölbýli                 Garðabær                new                      8.458583e+07   
                                                 old                      9

In [367]:
df_listings\
.loc[lambda r: r.square_meters < 300]\
.loc[lambda r: r.realestate_category_name == 'Fjölbýli']\
.assign(sqm_price = lambda r: r.price / r.square_meters)\
.groupby(['realestate_category_name', 'city_name', 'construction_category'])\
.agg(
    square_meters = ('square_meters', 'mean'),
    rooms = ('rooms', 'mean'),
    construction_year = ('construction_year', 'mean'),
    price = ('price', 'mean'),
    total = ('realestate_id', 'count'),
    sqm_price = ('sqm_price', 'mean'),
    days_listed = ('days_listed', 'mean')
)

square_meters  \
realestate_category_name city_name               construction_category                  
Fjölbýli                 Garðabær                new                       124.804630   
                                                 old                       120.806452   
                         Hafnarfjarðarkaupstaður new                        99.684987   
                                                 old                        98.370098   
                         Kópavogsbær             new                       105.545536   
                                                 old                       114.560909   
                         Mosfellsbær             new                        93.566667   
                                                 old                       102.468421   
                         Reykjavíkurborg         new                       105.063288   
                                                 old                       105.060265   
                         Seltjarnarnesbær        new                       116.186667   
                                                 old                       105.643750   

                                                                           rooms  \
realestate_category_name city_name               construction_category             
Fjölbýli                 Garðabær                new                    3.250000   
                                                 old                    3.362903   
                         Hafnarfjarðarkaupstaður new                    3.122137   
                                                 old                    3.294118   
                         Kópavogsbær             new                    3.312500   
                                                 old                    3.386364   
                         Mosfellsbær             new                    3.000000   
                                                 old                    3.421053   
                         Reykjavíkurborg         new                    3.030137   
                                                 old                    3.320574   
                         Seltjarnarnesbær        new                    3.333333   
                                                 old                    3.062500   

                                                                        construction_year  \
realestate_category_name city_name               construction_category                      
Fjölbýli                 Garðabær                new                          2024.731481   
                                                 old                          2010.991935   
                         Hafnarfjarðarkaupstaður new                          2023.870229   
                                                 old                          2001.397059   
                         Kópavogsbær             new                          2024.660714   
                                                 old                          1999.136364   
                         Mosfellsbær             new                          2024.000000   
                                                 old                          2014.736842   
                         Reykjavíkurborg         new                          2024.369863   
                                                 old                          1980.009514   
                         Seltjarnarnesbær        new                          2024.466667   
                                                 old                          1988.062500   

                                                                               price  \
realestate_category_name city_name               construction_category                 
Fjölbýli                 Garðabær                new                    1.262280e+08   
                                                 old                    1.047361e